In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"CUDA device name: {torch.cuda.get_device_name()}")
else:
    print("CUDA is not available.")

CUDA device name: NVIDIA GeForce GTX 1650 Ti with Max-Q Design


In [ ]:
pip install emoji --upgrade

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install nltk

In [ ]:
!pip install contractions

In [ ]:
import pandas as pd
import re
import emoji

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.util import mark_negation
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')

import contractions

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\pierr\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pierr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pierr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pierr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Loading the data
train_path = "twitter_training.csv"
val_path = "twitter_validation.csv"
# Note: ideally data doesnt have column headers but we add them to easy manipulation
headers = ["id", "company", "sentiment", "tweet"]
df_train = pd.read_csv(train_path, names=headers)
df_val = pd.read_csv(val_path, names=headers)

print(df_train.head())
print(df_val.head())

     id      company sentiment  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

                                               tweet  
0  im getting on borderlands and i will murder yo...  
1  I am coming to the borders and I will kill you...  
2  im getting on borderlands and i will kill you ...  
3  im coming on borderlands and i will murder you...  
4  im getting on borderlands 2 and i will murder ...  
     id    company   sentiment  \
0  3364   Facebook  Irrelevant   
1   352     Amazon     Neutral   
2  8312  Microsoft    Negative   
3  4371      CS-GO    Negative   
4  4433     Google     Neutral   

                                               tweet  
0  I mentioned on Facebook that I was struggling ...  
1  BBC News - Amazon boss Jeff Bezos rejects clai...  
2  @Microsoft Why do I pay for WORD when it funct...  
3  CSGO matchmaking is so full of closet

In [ ]:
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk",
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart",
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet",
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously",
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

In [ ]:
# Step 1: Remove NaN values
def remove_nan_values(df, columns):
    for column in columns:
        df = df[df[column].notna()]  # Remove rows with NaN values
        df = df[df[column].astype(bool)]  # Remove rows with empty lists or lists with only white spaces
    return df

# Step 2: Assign tweet labels
def one_hot_encode_sentiments(sentiment): #make neutral and irrelavent same number, and look over
    if sentiment == 'Positive':
        return 2
    elif sentiment == 'Negative':
        return 0
    elif sentiment == 'Neutral':
        return 1
    elif sentiment == 'Irrelevant':
        return 1

def expand_slang(text):
    tokens = word_tokenize(text)
    tokens = [abbreviations.get(word.lower(), word) for word in tokens]
    text = ' '.join(tokens)
    return text


# Step 3: Clean tweets
def clean_tweets(tweet):
    # Remove @mentions
    tweet = re.sub(r'@\w+', '', tweet)

    # Remove URLs
    tweet = re.sub(r'http\S+|www\S+', '', tweet)

    # Remove hashtags
    tweet = re.sub(r'#\w+', '', tweet)

    # Convert emojis to text
    tweet = emoji.demojize(tweet)

    # Expand contractions
    tweet = contractions.fix(tweet)
    ##############################3

    # Remove numbers
    tweet = re.sub(r'\d+', '', tweet)

    # Remove punctuation
    tweet = re.sub(r'[^\w\s]', '', tweet)

    # Remove excessive whitespace
    tweet = re.sub(r'\s+', ' ', tweet)

    # Standardize repeated characters
    tweet = re.sub(r'(.)\1+', r'\1\1', tweet)

    #Expand slang terms
    tweet = expand_slang(tweet)

    # Tokenize the tweet
    tokens = word_tokenize(tweet)

    # Lowercase and normalize
    tokens = [token.lower() for token in tokens]

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]

    # Handle abbreviations and acronyms
    abbreviation_words = {"lol": "laughing out loud", "btw": "by the way"}  # Add more as needed
    tokens = [abbreviation_words.get(token.lower(), token) for token in tokens]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Handle negation using mark_negation
    #tokens = mark_negation(tokens)
    #tweet = "I don't like this product. It is not good."
    #['I', "don't", 'like_NEG', 'this_NEG', 'product_NEG', '.', 'It_NEG', 'is_NEG', 'not_NEG', 'good_NEG', '.']

    # Uncomment the following lines to include POS tags as comments
    #pos_tags = nltk.pos_tag(tokens)
    #tokens_with_pos = [f"{token}/{pos}" for token, pos in pos_tags]

    return tokens

In [ ]:
# Step 1: Remove NaN values
columns_to_check = ["id", "company", "sentiment", "tweet"]
df_train = remove_nan_values(df_train, columns_to_check)
df_val = remove_nan_values(df_val, columns_to_check)

print(df_train.head())
print(df_val.head())

     id      company sentiment  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

                                               tweet  
0  im getting on borderlands and i will murder yo...  
1  I am coming to the borders and I will kill you...  
2  im getting on borderlands and i will kill you ...  
3  im coming on borderlands and i will murder you...  
4  im getting on borderlands 2 and i will murder ...  
     id    company   sentiment  \
0  3364   Facebook  Irrelevant   
1   352     Amazon     Neutral   
2  8312  Microsoft    Negative   
3  4371      CS-GO    Negative   
4  4433     Google     Neutral   

                                               tweet  
0  I mentioned on Facebook that I was struggling ...  
1  BBC News - Amazon boss Jeff Bezos rejects clai...  
2  @Microsoft Why do I pay for WORD when it funct...  
3  CSGO matchmaking is so full of closet

In [ ]:
# Step 2: Assign tweet labels and remove rows with 'Irrelevant' sentiment
df_train['sentiment'] = df_train['sentiment'].apply(one_hot_encode_sentiments)
df_val['sentiment'] = df_val['sentiment'].apply(one_hot_encode_sentiments)

# Remove rows with 'Irrelevant' sentiment from the DataFrame
df_train = df_train.dropna(subset=['sentiment'])
df_val = df_val.dropna(subset=['sentiment'])

print(df_train.head())
print(df_val.head())

     id      company  sentiment  \
0  2401  Borderlands          2   
1  2401  Borderlands          2   
2  2401  Borderlands          2   
3  2401  Borderlands          2   
4  2401  Borderlands          2   

                                               tweet  
0  im getting on borderlands and i will murder yo...  
1  I am coming to the borders and I will kill you...  
2  im getting on borderlands and i will kill you ...  
3  im coming on borderlands and i will murder you...  
4  im getting on borderlands 2 and i will murder ...  
     id    company  sentiment  \
0  3364   Facebook          1   
1   352     Amazon          1   
2  8312  Microsoft          0   
3  4371      CS-GO          0   
4  4433     Google          1   

                                               tweet  
0  I mentioned on Facebook that I was struggling ...  
1  BBC News - Amazon boss Jeff Bezos rejects clai...  
2  @Microsoft Why do I pay for WORD when it funct...  
3  CSGO matchmaking is so full of closet

In [ ]:
# Step 3: Clean tweets
df_train['tweet'] = df_train['tweet'].apply(clean_tweets)
df_val['tweet'] = df_val['tweet'].apply(clean_tweets)

print(df_train.head())
print(df_val.head())

     id      company  sentiment                          tweet
0  2401  Borderlands          2  [getting, borderland, murder]
1  2401  Borderlands          2         [coming, border, kill]
2  2401  Borderlands          2    [getting, borderland, kill]
3  2401  Borderlands          2   [coming, borderland, murder]
4  2401  Borderlands          2  [getting, borderland, murder]
     id    company  sentiment  \
0  3364   Facebook          1   
1   352     Amazon          1   
2  8312  Microsoft          0   
3  4371      CS-GO          0   
4  4433     Google          1   

                                               tweet  
0  [mentioned, facebook, struggling, motivation, ...  
1  [british, broadcasting, corporation, news, ama...  
2  [pay, word, function, poorly, chromebook, face...  
3  [csgo, matchmaking, full, closet, hacking, tru...  
4  [president, slapping, american, face, really, ...  


In [ ]:
# Step 1: Remove NaN values
columns_to_check = ["id", "company", "sentiment", "tweet"]
df_train = remove_nan_values(df_train, columns_to_check)
df_val = remove_nan_values(df_val, columns_to_check)

print(df_train.head())
print(df_val.head())

     id      company  sentiment                          tweet
0  2401  Borderlands          2  [getting, borderland, murder]
1  2401  Borderlands          2         [coming, border, kill]
2  2401  Borderlands          2    [getting, borderland, kill]
3  2401  Borderlands          2   [coming, borderland, murder]
4  2401  Borderlands          2  [getting, borderland, murder]
      id               company  sentiment  \
0   3364              Facebook          1   
1    352                Amazon          1   
4   4433                Google          1   
6   7925             MaddenNFL          2   
7  11332  TomClancysRainbowSix          2   

                                               tweet  
0  [mentioned, facebook, struggling, motivation, ...  
1  [british, broadcasting, corporation, news, ama...  
4  [president, slapping, american, face, really, ...  
6  [thank, new, te, austin, hooper, orange, brown...  
7  [rocket, league, sea, thief, rainbow, six, sie...  


In [ ]:
# Print the resulting DataFrame
print(df_train.head())
print(df_val.head())

     id      company  sentiment                          tweet
0  2401  Borderlands          2  [getting, borderland, murder]
1  2401  Borderlands          2         [coming, border, kill]
2  2401  Borderlands          2    [getting, borderland, kill]
3  2401  Borderlands          2   [coming, borderland, murder]
4  2401  Borderlands          2  [getting, borderland, murder]
      id               company  sentiment  \
0   3364              Facebook          1   
1    352                Amazon          1   
4   4433                Google          1   
6   7925             MaddenNFL          2   
7  11332  TomClancysRainbowSix          2   

                                               tweet  
0  [mentioned, facebook, struggling, motivation, ...  
1  [british, broadcasting, corporation, news, ama...  
4  [president, slapping, american, face, really, ...  
6  [thank, new, te, austin, hooper, orange, brown...  
7  [rocket, league, sea, thief, rainbow, six, sie...  


In [ ]:
# Assuming your DataFrame is named df
import torch
# Drop the 'id' column
df_train_new = df_train.drop('id', axis=1)
df_val_new = df_val.drop('id', axis=1)

# Export the DataFrame to a CSV file
df_train_new.to_csv('df_train_new_int.csv', index=False)
df_val_new.to_csv('df_val_new_int.csv', index=False)

In [ ]:
print(df_train_new[:5])

       company  sentiment                          tweet
0  Borderlands          2  [getting, borderland, murder]
1  Borderlands          2         [coming, border, kill]
2  Borderlands          2    [getting, borderland, kill]
3  Borderlands          2   [coming, borderland, murder]
4  Borderlands          2  [getting, borderland, murder]


In [ ]:
import torch
import pandas as pd
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset

# Load GloVe embeddings
def load_glove_embeddings(embeddings_path):
    embeddings_index = {}
    with open(embeddings_path, "r", encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = torch.tensor([float(val) for val in values[1:]])
            embeddings_index[word] = coefs
    return embeddings_index

def get_vocab_dict(word_counts):
    sorted_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
    vocab_dict = {word: idx for idx, (word, _) in enumerate(sorted_words)}
    return vocab_dict

def indices_to_embeddings(indices, embeddings_index, embedding_dim, vocab_dict):
    embeddings = []
    for idx in indices:
        word = list(vocab_dict.keys())[list(vocab_dict.values()).index(idx)]
        if word in embeddings_index:
            embeddings.append(embeddings_index[word])
        else:
            embeddings.append(torch.zeros(embedding_dim))
    return embeddings

# Paths
glove_path = "glove.twitter.27B.100d.txt"  # Replace with your actual path
data_path_train = "df_train_new_int.csv"  # Replace with your actual path
data_path_valid = "df_val_new_int.csv"  # Replace with your actual path

# Load data
df_train_new = pd.read_csv(data_path_train)
df_val_new = pd.read_csv(data_path_valid)

# Flatten the list of lists into a single list
all_words = [word for sublist in df_train_new['tweet'] for word in sublist]
all_words_valid = [word for sublist in df_val_new['tweet'] for word in sublist]

# Create a vocabulary dictionary with words ordered by frequency
word_counts = Counter(all_words + all_words_valid)
vocab_dict = get_vocab_dict(word_counts)

# Convert words to indices for each tweet
sequences_of_indices_train = [[vocab_dict[word] for word in sublist] for sublist in df_train_new['tweet']]
sequences_of_indices_valid = [[vocab_dict[word] for word in sublist] for sublist in df_val_new['tweet']]

# Load GloVe embeddings
glove_embeddings = load_glove_embeddings(glove_path)

# Convert indices to GloVe embeddings
embedding_dim = 100  # Assuming you're using 100-dimensional GloVe embeddings

train_embeddings = [indices_to_embeddings(seq, glove_embeddings, embedding_dim, vocab_dict) for seq in sequences_of_indices_train]
valid_embeddings = [indices_to_embeddings(seq, glove_embeddings, embedding_dim, vocab_dict) for seq in sequences_of_indices_valid]

# Ensure train_embeddings and valid_embeddings are lists of lists of tensors
train_embeddings = [torch.stack(embedding_list) for embedding_list in train_embeddings]
valid_embeddings = [torch.stack(embedding_list) for embedding_list in valid_embeddings]

max_train_seq_length = 100

# Pad sequences and create tensors
padded_train_embeddings = pad_sequence(
    [torch.cat((seq, torch.zeros(max_train_seq_length - len(seq), embedding_dim))) if len(seq) < max_train_seq_length else seq[:max_train_seq_length] for seq in train_embeddings],
    batch_first=True, padding_value=0.0
)

padded_valid_embeddings = pad_sequence(
    [torch.cat((seq, torch.zeros(max_train_seq_length - len(seq), embedding_dim))) if len(seq) < max_train_seq_length else seq[:max_train_seq_length] for seq in valid_embeddings],
    batch_first=True, padding_value=0.0
)

# Convert labels to tensors
train_labels = torch.tensor(df_train_new['sentiment'].values, dtype=torch.long)
valid_labels = torch.tensor(df_val_new['sentiment'].values, dtype=torch.long)

# Create datasets and data loaders
batch_size = 32
train_dataset = TensorDataset(padded_train_embeddings, train_labels)
valid_dataset = TensorDataset(padded_valid_embeddings, valid_labels)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

# Print shapes for verification
print("Train dataset shape:", padded_train_embeddings.shape)
print("Valid dataset shape:", padded_valid_embeddings.shape)

Train dataset shape: torch.Size([50380, 100, 100])
Valid dataset shape: torch.Size([734, 100, 100])


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import confusion_matrix
import numpy

class SentimentGRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_prob=0.0):
        super(SentimentGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Define the bidirectional GRU layer with batch normalization
        self.gru = nn.GRU(embedding_dim, hidden_size, num_layers, batch_first=True, dropout=dropout_prob, bidirectional=True)
        self.batch_norm = nn.BatchNorm1d(hidden_size * 2)  # Batch normalization layer

        # Define the fully connected layer for sentiment prediction
        self.fc = nn.Linear(hidden_size * 2, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)

        x = x.to(self.gru.weight_ih_l0.dtype)

        # Permute dimensions to (batch_size, input_size, sequence_length)
        x = x.permute(0, 2, 1)

        out, _ = self.gru(x, h0)
        out = self.batch_norm(out.permute(0, 2, 1)).permute(0, 2, 1)  # Apply batch normalization

        out = out[:, -1, :]

        out = self.fc(out)

        return out

def train(model, train_dataloader, learning_rate, epochs, optimizer):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = optimizer
    loss_fn = nn.BCEWithLogitsLoss()

    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"CUDA device name: {torch.cuda.get_device_name()}")
    else:
        print("CUDA is not available.")

    for epoch in range(epochs):
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        for batch in train_dataloader:
            inputs = batch[0].to(device)
            labels = batch[1].to(device).float()

            # Permute dimensions to (batch_size, input_size, sequence_length)
            inputs = inputs.permute(0, 2, 1)

            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = outputs.view(-1, 1)

            labels = labels.view(-1, 1)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            predicted_probs = torch.sigmoid(outputs)
            predicted_classes = (predicted_probs > 0.5).float()

            correct_predictions += (predicted_classes == labels).sum().item()
            total_predictions += labels.size(0)

        training_accuracy = correct_predictions / total_predictions
        epoch_loss = running_loss / len(train_dataloader)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss}, Accuracy: {training_accuracy * 100}%")

    return training_accuracy

def evaluate(model, dataloader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    model.eval()  # Set the model to evaluation mode
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for batch in dataloader:
            inputs = batch[0].to(device)
            labels = batch[1].to(device).float()

            # Permute dimensions to (batch_size, input_size, sequence_length)
            inputs = inputs.permute(0, 2, 1)

            outputs = model(inputs)
            predicted_probs = torch.sigmoid(outputs)
            predicted_classes = (predicted_probs > 0.5).float()

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted_classes.cpu().numpy())

    confusion = confusion_matrix(all_labels, all_predictions)
    return confusion

In [ ]:
compare = SentimentGRU(embedding_dim, 128, 3, 0.8)
train(compare, train_loader, 5e-05, 5, torch.optim.Adam(compare.parameters(), lr=5e-05))

confusion_matrix = evaluate(compare, valid_loader)
print("Confusion Matrix:")
print(confusion_matrix)

CUDA device name: NVIDIA GeForce GTX 1650 Ti with Max-Q Design
Epoch [1/5], Loss: -77.04780857339738, Accuracy: 59.90075426756649%
Epoch [2/5], Loss: -199.42873069157676, Accuracy: 60.043668122270745%
Epoch [3/5], Loss: -327.04808712913876, Accuracy: 60.043668122270745%
Epoch [4/5], Loss: -470.2974644252232, Accuracy: 60.043668122270745%
Epoch [5/5], Loss: -631.2811776684957, Accuracy: 60.043668122270745%
Confusion Matrix:
[[457   0]
 [277   0]]


In [ ]:
pip install hyperopt

In [ ]:
from hyperopt import hp, fmin, tpe, Trials

# Define the hyperparameter search space
space = {
    'lr': hp.choice('lr', [1e-5, 1e-4, 1e-3, 1e-2, 5e-5, 5e-4]),
    'hidden_size': hp.choice('hidden_size', [32, 64, 128, 256]),
    'num_layers': hp.choice('num_layers', [1, 2, 3]),
    'dropout': hp.choice('dropout', [0.0, 0.2,0.35, 0.5, 0.75,0.8,1]),  # Dropout rate
    'optimizer': hp.choice('optimizer', ['adam', 'sgd', 'rmsprop']),  # Optimizer choice
    'epochs': 2  # Run each model for 1 epoch
}

# Initialize a list to store the top 10 hyperparameter configurations and their accuracies
top_configs = []

# Define the objective function to optimize (in this case, the negative accuracy)
def objective(params):
    lr = params['lr']
    hidden_size = params['hidden_size']
    num_layers = params['num_layers']
    dropout = params['dropout']
    optimizer_choice = params['optimizer']
    epochs = params['epochs']

    # Create the model with the given hyperparameters
    model = SentimentRNN(198, hidden_size, num_layers, dropout)

    # Choose optimizer
    optimizer = None
    if optimizer_choice == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    elif optimizer_choice == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    elif optimizer_choice == 'rmsprop':
        optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)

    # Train the model and get the accuracy
    accuracy = train(model, train_loader, lr, epochs, optimizer)

    # Add the hyperparameters and accuracy to the top_configs list
    top_configs.append({'lr': lr, 'hidden_size': hidden_size, 'num_layers': num_layers,
                        'dropout': dropout, 'optimizer': optimizer_choice,
                        'accuracy': accuracy})

    # Since hyperopt minimizes the objective function, we need to return the negative accuracy
    return -accuracy

# Initialize the Trials object to track the optimization process
trials = Trials()

# Run the hyperparameter search using the Tree of Parzen Estimators (TPE) algorithm
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials)

# Sort the top_configs list based on accuracy in descending order
top_configs.sort(key=lambda x: x['accuracy'], reverse=True)

# Print the 10 best hyperparameter configurations and their accuracies
for i, config in enumerate(top_configs[:10]):
    print(f"Top {i+1} Hyperparameters:")
    print(f"Learning Rate: {config['lr']}")
    print(f"Hidden Layer Size: {config['hidden_size']}")
    print(f"Number of Hidden Layers: {config['num_layers']}")
    print(f"Dropout Rate: {config['dropout']}")
    print(f"Optimizer: {config['optimizer']}")
    print(f"Accuracy: {config['accuracy']}\n")

Epoch [1/2], Loss: -66.68634116216312, Accuracy: 59.960301707026595%
  0%|          | 0/100 [02:39<?, ?trial/s, best loss=?]


KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt
train_path = "sentiment-emotion-labelled_Dell_tweets.csv"
# use pathlib to load path
df = pd.read_csv(train_path, error_bad_lines=False)

fig = plt.figure(figsize=(8,5))
ax = sns.barplot(x=df.sentiment.unique(),y=df.sentiment.value_counts());
ax.set(xlabel='Labels')
# Note: The label distribution is not even!